In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from copy import deepcopy
import numpy as np

from bio.strings import NUC2INT

profile = np.array([
    [.4, .2, .2, .2],
    [0, .6, .2, 0],
    [.2, .2, .8, 0],
    [0, .2, 0, .8]
])

Dna = [
    "ttaccttaac",
    "gatgtctgtc",
    "acggcgttag",
    "ccctaacgag",
    "cgtcagaggt"
    ]
Dna = [dna.upper() for dna in Dna]

from dataclasses import dataclass
from bio.motifs import most_probable_kmer_from_profile

@dataclass
class Motifs:
    kmers: list[str]
    profile_with_pseudocount: bool = False

    def __post_init__(self) -> None:
        self.kmers = [kmer.upper() for kmer in self.kmers] 
        self.k = len(self.kmers[0])
        self.t = len(self.kmers)
        self.profile = self.build_profile(self.profile_with_pseudocount)
        

    def __get_item__(self, idx: int) -> str:
        return self.kmers[idx]
    
    @property
    def score(self) -> int:
        """scores motifs distance to consensus string"""
        distance_to_consensus = 1 - self.profile.max(axis=0)
        return int(distance_to_consensus.sum() * self.t)
    
    # def consensus(self) -> str:

    def build_profile(self, with_pseudocounts: bool = False) -> np.ndarray:
        """creates a frequency-based probability matrix to find each nucleotide (rows) for each position of the k-mer (columns)"""
        profile = np.zeros((4, self.k))
        t = self.t
        for motif in self.kmers:
            for i, c in enumerate(motif):
                profile[NUC2INT[c], i] += 1
        if with_pseudocounts:
            # adding 1 count to each nucleic acid to avoid zero probabilities
            profile += 1
            t += 4
        return profile / t

def motifs_from_profile(profile: np.ndarray, Dna: list[str]) -> Motifs: 
    k = profile.shape[1]
    return Motifs([most_probable_kmer_from_profile(dna, k, profile) for dna in Dna])

import random

def randomized_motif_search(k: int, t: int, Dna: list[str]) -> Motifs:
    # random initialization
    starting_indices = [random.randint(0, t - k) for _ in range(t)]
    motifs = Motifs([Dna[i][j: j+k] for i, j in enumerate(starting_indices)])
    best_motifs = deepcopy(motifs)
    
    while True:
        motifs = motifs_from_profile(motifs.profile, Dna)
        if motifs.score < best_motifs.score:
            best_motifs = deepcopy(motifs)
        else:
            return best_motifs


randomized_motif_search(k=4, t=5, Dna=Dna)


['TTAC', 'GTCT', 'CGGC', 'CCCT', 'GTCA']

In [12]:
m2.build_profile()

array([[0.8, 0. , 0. , 0.2],
       [0. , 0.6, 0.2, 0. ],
       [0.2, 0.2, 0.8, 0. ],
       [0. , 0.2, 0. , 0.8]])

In [19]:
def normalize(ls: list[str]) -> list[str]:
    return [s.upper() for s in ls]
Dna = [
    "ttaccttaac", 
    "gaTGTctgtc", 
    "ccggcgttag", 
    "cactaacgag", 
    "cgtcagaggt"]

m3 = Motifs(["taac", "gtct", "ccgg", "acta", "aggt"])
m3.profile

array([[0.4, 0.2, 0.2, 0.2],
       [0.2, 0.4, 0.2, 0.2],
       [0.2, 0.2, 0.4, 0.2],
       [0.2, 0.2, 0.2, 0.4]])

In [23]:
args = {'k': 8, 't': 5, 'Dna': 'CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA'}
randomized_motif_search(**args)

ValueError: empty range for randrange() (0, -2, -2)

In [20]:
motifs_from_profile(m3.profile, normalize(Dna))

Motifs(kmers=['ACCT', 'ATGT', 'GCGT', 'ACGA', 'AGGT'], profile_with_pseudocount=False)

In [1]:
from bio.motifs import greedy_motif_search, greedy_motif_search_v2, score, build_profile

args = {'k': 3, 't': 5, 'Dna': ['GGCGTTCAGGCA', 'AAGAATCAGTCA', 'CAAGGAGTTCGC', 'CACGTCAATCAC', 'CAATAATATTCG']}

motifs = greedy_motif_search_v2(**args, with_pseudocounts=True)
motifs.profile



0 20
1 20
2 15
3 15
4 10
5 10
6 10
7 15
8 20
9 25


<bound method Motifs.profile of Motifs(kmers=['TTC', 'ATC', 'TTC', 'ATC', 'TTC'])>

In [2]:
v1 = greedy_motif_search(**args, with_pseudocounts=True)
v2 = greedy_motif_search_v2(**args, with_pseudocounts=True)
v1, v2.kmers

0 3
1 4
2 3
3 3
4 2
5 2
6 2
7 3
8 3
9 5
0 20
1 20
2 15
3 15
4 10
5 10
6 10
7 15
8 20
9 25


(['TTC', 'ATC', 'TTC', 'ATC', 'TTC'], ['TTC', 'ATC', 'TTC', 'ATC', 'TTC'])

In [1]:
from bio.motifs import Motifs

motifs = Motifs(['TTC', 'ATC', 'TTC', 'ATC', 'TTC'])
motifs.score

array([2., 0., 0.])

In [3]:
assert v2.kmers == v1
assert (build_profile(v1, with_pseudocounts=True) == v2.profile).all()

AssertionError: 

In [7]:
v2.counts, v2.score

(array([[2., 0., 0.],
        [0., 0., 5.],
        [0., 0., 0.],
        [3., 5., 0.]]),
 10)

In [10]:
distance_to_consensus = v2.t - v2.counts.max(axis=0)
distance_to_consensus

array([2., 0., 0.])

In [17]:
v2.consensus()

AttributeError: 'Motifs' object has no attribute 'consensus'

In [1]:
from bio.motifs import Motifs

motif_exp = Motifs("TCTCGGGG CCAAGGTG TACAGGCG TTCAGGTG TCCACGTG".split())
motif_wrong = Motifs("AACGGCCA AAGTGCCA TAGTACCG AAGTTTCA ACGTGCAA".split())

motif_exp.score, motif_wrong.score

(np.float64(9.0), np.float64(9.0))